In [1]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import date, timedelta
from pandas.tseries.offsets import *
import calendar

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"

today = date.today()
yesterday = today - timedelta(days=1)
today, yesterday

(datetime.date(2023, 6, 2), datetime.date(2023, 6, 1))

In [2]:
#today = date(2023, 5, 12)
today

datetime.date(2023, 6, 2)

### Today must be last business day

In [3]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
print(f'today: {today}')
to_mmdd = today.strftime('%m-%d')
yesterday = yesterday.date()
print(f'Today in mm-dd format: {to_mmdd}')
print(f'yesterday: {yesterday}')

today: 2023-06-02
Today in mm-dd format: 06-02
yesterday: 2023-06-01


In [4]:
format_dict = {    
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}','qtrly':'{:.4f}',
    'price':'{:.2f}',    
    'amount':'{:,.2f}','net':'{:,.2f}','cost_amt':'{:,.2f}',
    'yield':'{:,.2f}%','pct':'{:,.2f}%',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
              }

### New dividend stock

In [4]:
name = 'WHAIR'
sql = """
SELECT * 
FROM DIVIDEND
WHERE name = '%s'
"""
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual


In [5]:
sqlIns = """
INSERT INTO dividend
VALUES('WHAIR',.1738,0.1738,0.1738,0.1556,0,0,0,00,'2022-05-25','2022-06-22','',0)
"""
rp = const.execute(sqlIns)
rp.rowcount

1

### Start of Update dividend

In [5]:
def update_dividend1(q1,XDATE,PAIDDATE,actual,name):
    sql = "UPDATE dividend SET q1 = %s, dividend = q1+q2+q3+q4, XDATE = '%s', PAIDDATE = '%s', actual = %s WHERE name = '%s'"
    sql = sql % (q1,XDATE, PAIDDATE, actual,name)
    rp = const.execute(sql)
    return "Records updated = " + str(rp.rowcount)

In [11]:
name = 'IVL'
sql = """
SELECT * FROM DIVIDEND WHERE name = '%s'"""
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,IVL,0.4000,0.4000,0.4000,0.2500,1.4500,"7,200",2023-05-26,2023-06-14,,1


In [10]:
q1 = 0.25
XDATE = '2023-05-26'
PAIDDATE = '2023-06-14'
actual = 1

update_dividend1(q1,XDATE,PAIDDATE,actual,name)

'Records updated = 1'

In [7]:
def update_dividend2(shares,q1,actual,name):
    sql = "UPDATE dividend SET shares = %s, q1 = %s, dividend=q1+q2+q3+q4, actual = %s WHERE name = '%s'"
    sql = sql % (shares, q1, actual, name)
    rp = const.execute(sql)
    return "Records updated = " + str(rp.rowcount)

In [8]:
name = 'STA'
sql = """
SELECT * 
FROM DIVIDEND
WHERE name = '%s'
"""
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,STA,0.6500,1.2500,1.2500,1.0000,4.1500,"20,000",2022-04-19,2022-05-06,,1


In [9]:
shares = 20000
q1 = 0
actual = 0

In [11]:
update_dividend2(shares,q1,actual,name)

'Records updated = 1'

### Toggle actual status

In [2]:
name = 'BCT'
actual = 0
sqlUpd = "UPDATE dividend SET actual = %s WHERE name = '%s'"
sqlUpd = sqlUpd % (actual, name)
rp = const.execute(sqlUpd)
rp.rowcount

1

### Delete dividend record

In [10]:
sqlDel = '''
DELETE FROM dividend
WHERE name IN ("MC")
'''
#rp = const.execute(sqlDel)
rp.rowcount

1

### Start of output to cloud

In [5]:
cols = 'name qtrly shares amount net xdate paiddate cost_amt pct actual'.split()

In [7]:
sql = """
SELECT Y.NAME AS name,  Q1 AS qtrly, SHARES, XDATE, PAIDDATE, 
P.price AS price, Y.DIVIDEND, ACTUAL, B.price * B.volbuy AS cost_amt
FROM dividend AS Y, price AS P, buy AS B
WHERE Y.name = P.name 
AND Y.name = B.name 
AND Q1 > 0 
AND P.date = '%s'
ORDER BY name
"""
sql = sql % yesterday 
print(sql)


SELECT Y.NAME AS name,  Q1 AS qtrly, SHARES, XDATE, PAIDDATE, 
P.price AS price, Y.DIVIDEND, ACTUAL, B.price * B.volbuy AS cost_amt
FROM dividend AS Y, price AS P, buy AS B
WHERE Y.name = P.name 
AND Y.name = B.name 
AND Q1 > 0 
AND P.date = '2023-06-01'
ORDER BY name



In [8]:
dividend = pd.read_sql(sql, const)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend['amount'] = round(dividend['shares'] * dividend['qtrly'], 2)
dividend['net'] = round(dividend['amount'] * 0.9, 2)
dividend['pct'] = round(dividend['net'] / dividend['cost_amt'] * 100, 2)
dividend[cols].style.format(format_dict)

,name,qtrly,shares,amount,net,xdate,paiddate,cost_amt,pct,actual
0,CPNREIT,0.2470,"60,000","14,820.00","13,338.00",2023-05-23,2023-06-08,"1,053,000.00",1.27%,1
1,DIF,0.2450,"30,000","7,350.00","6,615.00",2023-05-15,2023-06-07,"441,000.00",1.50%,1
2,GVREIT,0.1915,"30,000","5,745.00","5,170.50",2023-02-27,2023-03-10,"267,000.00",1.94%,0
3,IVL,0.4000,"7,200","2,880.00","2,592.00",2023-05-03,2023-05-19,"288,000.00",0.90%,0
4,JASIF,0.2300,"130,000","29,900.00","26,910.00",2023-05-19,2023-06-07,"1,300,000.00",2.07%,1
5,TFFIF,0.1032,"20,000","2,064.00","1,857.60",2023-05-31,2023-06-19,"153,000.00",1.21%,1
6,WHAIR,0.1369,"50,000","6,845.00","6,160.50",2023-05-22,2023-06-22,"435,000.00",1.42%,1
7,WHART,0.1915,"30,000","5,745.00","5,170.50",2023-05-23,2023-06-09,"351,000.00",1.47%,1


In [9]:
dividend.net.sum()

67814.1

In [25]:
file_name = 'dividend-q1.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
osd_file = osd_path + file_name
one_file = one_path + file_name

dividend[cols].sort_values(['name'],ascending=[True]).to_csv(output_file, index=False)
dividend[cols].sort_values(['name'],ascending=[True]).to_csv(data_file, index=False)
dividend[cols].sort_values(['name'],ascending=[True]).to_csv(osd_file, index=False)
dividend[cols].sort_values(['name'],ascending=[True]).to_csv(one_file, index=False)

In [10]:
file_name = 'dividend-q1.xlsx'
data_file = data_path + file_name
output_file = csv_path + file_name
osd_file = osd_path + file_name
one_file = one_path + file_name

dividend[cols].sort_values(['name'],ascending=[True]).to_excel(output_file, index=False)
dividend[cols].sort_values(['name'],ascending=[True]).to_excel(data_file, index=False)
dividend[cols].sort_values(['name'],ascending=[True]).to_excel(osd_file, index=False)
dividend[cols].sort_values(['name'],ascending=[True]).to_excel(one_file, index=False)

### End of output to cloud